In [12]:
import json
import random
from typing import TypedDict

import sqlalchemy
from google.cloud.sql.connector.connector import Connector

PROJECT_ID = "rl-llm-dev"
REGION = "us-central1"
INSTANCE_NAME = "csm-database"
# i.e demo-project:us-central1:demo-instance
INSTANCE_CONNECTION_NAME = f"{PROJECT_ID}:{REGION}:{INSTANCE_NAME}"
print(f"Your instance connection name is: {INSTANCE_CONNECTION_NAME}")
DB_USER = "root"
DB_PASS = 'f"OIl>#{R|[(^$"H'
DB_NAME = "csm"

# initialize Connector object
connector = Connector()

# function to return the database connection object
def getconn():
    """Funtion to return the dabase connection Object.

    Returns:
        Cloud Sql connection object

    """
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pymysql",
        user=DB_USER,
        password=DB_PASS,
        db=DB_NAME,
    )
    return conn

# create connection pool with 'creator' argument to our connection object function
pool = sqlalchemy.create_engine(
    "mysql+pymysql://",
    creator=getconn,
)

Your instance connection name is: rl-llm-dev:us-central1:csm-database


In [13]:
sql_statement = "UPDATE products SET price = :price, quantity = :quantity WHERE id = :id"

In [14]:
with open("./recommendation_products.jsonl") as f:
    lines = f.readlines()
    lines = [json.loads(i) for i in lines]

In [15]:
# connect to connection pool
with pool.connect() as db_conn:
    # create ratings table in our sandwiches database
    # db_conn.execute(sqlalchemy.text(create_sql))

    # commit transaction (SQLAlchemy v2.X.X is commit as you go)
    # db_conn.commit()
    # insert data into our ratings table
    update_stmt = sqlalchemy.text(sql_statement)

    for i in lines:
        update_dict = {
            "price": float(i["priceInfo"]["price"]),
            "quantity": i["availableQuantity"],
            "id": int(i["id"])
        }

        # insert entries into table
        db_conn.execute(update_stmt, parameters=update_dict)

    # commit transactions
    db_conn.commit()        